In [ ]:
# kbo_schedule_2001_2025.csv db에 맞게 재가공
import pandas as pd
df = pd.read_csv("games.csv", encoding="utf-8-sig")
# 1. 연도와 날짜를 문자열로 합치기
df["GAME_DATE"] = df["연도"].astype(str) + "-" + df["날짜"].astype(str)

# 2. datetime 타입으로 변환
df["GAME_DATE"] = pd.to_datetime(df["GAME_DATE"], format="%Y-%m.%d")

# 3. 경기 컬럼 데이터에서 'vs' 기준으로 필요한 컬럼에 데이터 담기
def parse_game_column(game_str):
    try:
        away, home = game_str.split("vs")

        # 팀명(isalpha는 글자만 true) .join → ['L', 'G'] → "LG"
        away_team_id = ''.join([a for a in away if a.isalpha()])
        home_team_id = ''.join([h for h in home if h.isalpha()])

        # 점수(isdigit는 숫자만 ture) .join -> ['1', '1'] → "11"
        away_score = ''.join([a for a in away if a.isdigit()])
        home_score = ''.join([h for h in home if h.isdigit()])

        # 숫자 문자열 → 실제 정수로 변환
        away_score = int(away_score) if away_score else None
        home_score = int(home_score) if home_score else None

        return pd.Series([away_team_id, home_team_id, away_score, home_score])
    except:
        return pd.Series([None, None, None, None]) 

# 4. csv에서 경기 컬럼 분해 → 팀명/점수 컬럼 생성
df[["AWAY_TEAM_NAME", "HOME_TEAM_NAME", "AWAY_SCORE", "HOME_SCORE"]] = df["경기"].apply(parse_game_column)

# 5. 컬럼 이름 변경
df = df.rename(columns={
    "시간": "GAME_TIME",
    "경기": "STATUS",
    "구장": "STADIUM",
    "비고": "IS_RAINED_OUT
})

# 6. 필요없는 컬럼 삭제
df = df.drop(columns=["연도", "날짜", "TV", "라디오"])

# 7. db에 맞게 컬럼 순서 변경
df = df[["GAME_DATE", "GAME_TIME", "STADIUM", "HOME_SCORE", "AWAY_SCORE", "IS_RAINED_OUT", "HOME_TEAM_NAME", "AWAY_TEAM_NAME"]]

# 8. IS_RAINED_OUT컬럼에 '우천취소'가 있으면 'Y', 나머지는 'N'
df["IS_RAINED_OUT"] = df["IS_RAINED_OUT"].apply(
    lambda x: 'Y' if str(x).strip() in ['우천취소'] else 'N'
)

# 9. 재가공된 파일 새로 저장
df.to_csv("GAMES_수정본.csv", index=False, encoding="utf-8-sig")

In [114]:
# 사라진 팀이 있어서 외래키 참조 오류땜에 숫자로 치환했던거 다시 이름으로 치환
# # 4. 팀명을 숫자로 치환
# team_name_to_id = {
#     9 : "한화", 5 : "LG", 2 : "롯데", 7 : "KT",
#     3 : "삼성", 8 : "SSG", 4 : "KIA", 1 : "NC",
#     6 : "두산", 10 : "키움", 11 : "고양", 
#     12 : "현대", 13 : "SK", 14 : "넥센" 
# }
# df["HOME_TEAM_ID"] = df["HOME_TEAM_ID"].replace(team_name_to_id)
# df["AWAY_TEAM_ID"] = df["AWAY_TEAM_ID"].replace(team_name_to_id)
# df

In [ ]:
# 타입 변경이 안돼서 엑셀에서 =TEXT(A2:A16348,"yyyy-mm-dd") & " " & TEXT(B2:B16348,"hh:mm:ss") 코드로 수정 후디비에서 파일 바로 넣음.

# # 2. Oracle 연결
# conn = get_connection()
# cursor = conn.cursor()

# # 3. INSERT 문 실행 (컬럼명 일치하는지 확인!)
# insert_sql = """
#     INSERT INTO GAMES (
#         GAME_DATE, GAME_TIME, STADIUM, HOME_SCORE, AWAY_SCORE, IS_RAINED_OUT, HOME_TEAM_NAME, AWAY_TEAM_NAME
#     ) VALUES (
#         TO_DATE(:1, 'YYYY-MM-DD'), TO_DATE(:2, 'HH24:MI:SS'), :3, :4, :5, :6, :7, :8
#     )
# """

# # 4. 데이터 삽입
# for i, row in df_db.iterrows():
#     try:
#         cursor.execute(insert_sql, (
#             row.GAME_DATE,
#             row.GAME_TIME,
#             row.STADIUM,
#             row.HOME_SCORE,
#             row.AWAY_SCORE,
#             row.IS_RAINED_OUT,
#             row.HOME_TEAM_NAME,
#             row.AWAY_TEAM_NAME
#         ))
#     except Exception as e:
#         print(f"[❌ 에러 발생] index={i}")
#         print(f"전체 값: {row.to_dict()}")
#         print(f"에러: {e}")
#         break


# # 5. 커밋 & 종료
# conn.commit()
# cursor.close()
# conn.close()